0. load from 

In [37]:
import pprint
import time

import pandas
import numpy

import plotly.express as pltxpr

import axis_and_allies.reinforcement_learning.super_simple_env as super_simple_env
import axis_and_allies.run_simulation as run_simulation
import axis_and_allies.combat as combat

In [16]:
input_armies = "possible_armies_IPC_30_r66x6.txt"
input_unit_dict = "../../unit_data.json"
N_sim = 1000

In [11]:
unit_dict = run_simulation.load_units(input_unit_dict)
pprint.pprint(unit_dict)

{'aircraft carrier': name:  aircraft carrier
id:  20
ipc:  14
attack:  1
temp_attack:  1
defense:  2
unit_type:  naval
move:  2
max_hit_points:  1
cur_hit_points:  1,
 'anti-aircraft artillery': name:  anti-aircraft artillery
id:  14
ipc:  5
attack:  0
temp_attack:  0
defense:  1
unit_type:  land
move:  1
max_hit_points:  1
cur_hit_points:  1,
 'artillery': name:  artillery
id:  12
ipc:  4
attack:  2
temp_attack:  2
defense:  2
unit_type:  land
move:  1
max_hit_points:  1
cur_hit_points:  1,
 'battleship': name:  battleship
id:  21
ipc:  20
attack:  4
temp_attack:  4
defense:  4
unit_type:  naval
move:  2
max_hit_points:  2
cur_hit_points:  2,
 'bomber': name:  bomber
id:  16
ipc:  12
attack:  4
temp_attack:  4
defense:  1
unit_type:  air
move:  6
max_hit_points:  1
cur_hit_points:  1,
 'cruiser': name:  cruiser
id:  19
ipc:  12
attack:  3
temp_attack:  3
defense:  3
unit_type:  naval
move:  2
max_hit_points:  1
cur_hit_points:  1,
 'destroyer': name:  destroyer
id:  18
ipc:  8
attack:

In [4]:
armies_df = pandas.read_csv(input_armies, sep="\t")
armies_df

,inf,art,tank,fight,bomb,cost
0,0,0,0,3,0,30
1,0,0,1,0,2,30
2,0,0,1,1,1,28
3,0,0,3,0,1,30
4,0,0,3,1,0,28
...,...,...,...,...,...,...
61,6,3,0,0,0,30
62,7,2,0,0,0,29
63,8,0,1,0,0,30
64,8,1,0,0,0,28


In [14]:
# ssenv = super_simple_env.SuperSimpleEnv(None, None, unit_dict)
# del ssenv

In [15]:
opponent_unit_names = ["infantry"]*2
print(opponent_unit_names)

['infantry', 'infantry']


In [24]:
sum_start_ipc_defense = run_simulation.calculate_sum_ipc(
    run_simulation.build_units_from_names(unit_dict, opponent_unit_names)
)
sum_start_ipc_defense

6

In [22]:
start_time = time.time()
prev_time = start_time

all_result_list = []
for row in armies_df.to_numpy():
    print("################")
    print(row)
    
    unit_names = super_simple_env.convert_unit_count_to_unit_name_list(row[:-1])
    print(unit_names)
    
    cur_result_list = []
    all_result_list.append(cur_result_list)
    for i in range(N_sim):
        result = run_simulation.run_single_from_names(unit_dict, unit_names, opponent_unit_names, combat.BATTLE_TYPE_LAND)
#         print(result[-1])
        cur_result_list.append(result[-1])
#         break
        cur_time = time.time()
        if (cur_time - prev_time) > 60.:
            print("progress:", i)
            prev_time = cur_time
            
#     break

end_time = time.time()
duration = end_time - start_time
print("duration:", duration, duration/60.)

print(len(all_result_list))

################
[ 0  0  0  3  0 30]
['fighter', 'fighter', 'fighter']
################
[ 0  0  1  0  2 30]
['tank', 'bomber', 'bomber']
################
[ 0  0  1  1  1 28]
['tank', 'fighter', 'bomber']
################
[ 0  0  3  0  1 30]
['tank', 'tank', 'tank', 'bomber']
################
[ 0  0  3  1  0 28]
['tank', 'tank', 'tank', 'fighter']
################
[ 0  0  5  0  0 30]
['tank', 'tank', 'tank', 'tank', 'tank']
################
[ 0  1  0  0  2 28]
['artillery', 'bomber', 'bomber']
################
[ 0  1  1  2  0 30]
['artillery', 'tank', 'fighter', 'fighter']
################
[ 0  1  2  0  1 28]
['artillery', 'tank', 'tank', 'bomber']
################
[ 0  1  4  0  0 28]
['artillery', 'tank', 'tank', 'tank', 'tank']
################
[ 0  2  0  1  1 30]
['artillery', 'artillery', 'fighter', 'bomber']
################
[ 0  2  0  2  0 28]
['artillery', 'artillery', 'fighter', 'fighter']
################
[ 0  2  2  1  0 30]
['artillery', 'artillery', 'tank', 'tank', 'fighter']

In [31]:
all_result_metric_list = []

for index in armies_df.index:
    print("###################", index)
    cur_result_list = all_result_list[index]
    sum_start_ipc_attack = armies_df.loc[index, "cost"]
    print(sum_start_ipc_attack)
    
    cur_result_metric_list = []
    all_result_metric_list.append(cur_result_metric_list)
    for cur_result in cur_result_list:
#         print(cur_result)
        cur_metric = run_simulation.calculate_metrics_from_combat_result(
            cur_result, sum_start_ipc_attack, sum_start_ipc_defense
        )
#         print(cur_metric)
        cur_result_metric_list.append(cur_metric)

print(len(all_result_metric_list))
print(set([len(x) for x in all_result_metric_list]))

################### 0
30
################### 1
30
################### 2
28
################### 3
30
################### 4
28
################### 5
30
################### 6
28
################### 7
30
################### 8
28
################### 9
28
################### 10
30
################### 11
28
################### 12
30
################### 13
30
################### 14
28
################### 15
30
################### 16
28
################### 17
28
################### 18
30
################### 19
30
################### 20
28
################### 21
29
################### 22
29
################### 23
29
################### 24
29
################### 25
29
################### 26
29
################### 27
29
################### 28
30
################### 29
28
################### 30
30
################### 31
28
################### 32
30
################### 33
30
################### 34
28
################### 35
28
################### 36
30
################### 37
30
################### 38

In [53]:
t = []
for i, cur_result_metric_list in enumerate(all_result_metric_list):
    df = pandas.DataFrame({
        "army_id":i,
        "fraction_ipc_winner":[x.fraction_ipc_winner for x in cur_result_metric_list]
    })
    t.append(df)

result_df = pandas.concat(t, axis=0)
result_df["army_id_str"] = result_df.army_id.astype(str)
result_df

,army_id,fraction_ipc_winner,army_id_str
0,0,0.666667,0
1,0,0.666667,0
2,0,0.666667,0
3,0,1.000000,0
4,0,0.333333,0
...,...,...,...
995,65,0.800000,65
996,65,1.000000,65
997,65,1.000000,65
998,65,1.000000,65


In [56]:
t = result_df[["army_id_str", "fraction_ipc_winner"]].groupby("army_id_str").median()
t.columns = ["median_fraction_ipc_winner"]

u = result_df[["army_id_str", "fraction_ipc_winner"]].groupby("army_id_str").min()
u.columns = ["min_fraction_ipc_winner"]

stat_result_df = t.join(u)
stat_result_df.sort_values(["median_fraction_ipc_winner", "min_fraction_ipc_winner"], inplace=True)
stat_result_df

,median_fraction_ipc_winner,min_fraction_ipc_winner
army_id_str,,
0,0.666667,-1.000000
2,0.785714,-1.000000
4,0.785714,-1.000000
1,0.800000,-1.000000
3,0.800000,-0.500000
...,...,...
41,0.900000,0.533333
49,0.900000,0.600000
53,0.900000,0.600000


In [59]:

fig = pltxpr.box(
    result_df, x="army_id_str", y="fraction_ipc_winner",
    points="all",
    category_orders={"army_id_str":stat_result_df.index}
)
# fig.show()
fig.write_html("temp.html", include_plotlyjs="cdn")

# fig = pltxpr.violin(
#     result_df, x="army_id", y="fraction_ipc_winner",
#     category_orders={"army_id":median_result_df.index}
# )
# fig.write_html("temp2.html", include_plotlyjs="cdn")

In [67]:
combined_df = armies_df.copy()
combined_df["army_id_str"] = combined_df.index.astype(str)
combined_df = combined_df.join(stat_result_df, on="army_id_str")
combined_df.sort_values(["median_fraction_ipc_winner", "min_fraction_ipc_winner"], inplace=True)
display(combined_df.head(10))
display(combined_df.tail(10))

,inf,art,tank,fight,bomb,cost,army_id_str,median_fraction_ipc_winner,min_fraction_ipc_winner
0,0,0,0,3,0,30,0,0.666667,-1.0
2,0,0,1,1,1,28,2,0.785714,-1.0
4,0,0,3,1,0,28,4,0.785714,-1.0
1,0,0,1,0,2,30,1,0.800000,-1.0
3,0,0,3,0,1,30,3,0.800000,-0.5
5,0,0,5,0,0,30,5,0.800000,0.4
6,0,1,0,0,2,28,6,0.857143,-1.0
11,0,2,0,2,0,28,11,0.857143,-1.0
8,0,1,2,0,1,28,8,0.857143,-0.5
9,0,1,4,0,0,28,9,0.857143,0.0


,inf,art,tank,fight,bomb,cost,army_id_str,median_fraction_ipc_winner,min_fraction_ipc_winner
59,6,0,2,0,0,30,59,0.9,0.500000
63,8,0,1,0,0,30,63,0.9,0.500000
65,10,0,0,0,0,30,65,0.9,0.500000
36,2,2,1,1,0,30,36,0.9,0.533333
37,2,3,0,0,1,30,37,0.9,0.533333
41,2,6,0,0,0,30,41,0.9,0.533333
49,4,0,3,0,0,30,49,0.9,0.600000
53,4,3,1,0,0,30,53,0.9,0.600000
61,6,3,0,0,0,30,61,0.9,0.600000
39,2,3,2,0,0,30,39,0.9,0.666667
